In [ ]:
%matplotlib inline

In [ ]:
import xarray as xr
from lib.thermo import liquid_water_temperature, get_dz, mass_integrate
from lib.advection import material_derivative

stat = xr.open_dataset("../data/raw/2/NG_5120x2560x34_4km_10s_QOBS_EQX/stat.nc")
fields_3d = xr.open_mfdataset("../data/raw/2/NG_5120x2560x34_4km_10s_QOBS_EQX/coarse/3d/*.nc")
fields_2d = xr.open_dataset("../data/raw/2/NG_5120x2560x34_4km_10s_QOBS_EQX/coarse/2d/all.nc")

# for some reason the time values are all scrambled up.
fields_2d = fields_2d.isel(time=fields_2d.time.values.argsort())
data = xr.merge((fields_2d, fields_3d), join='inner').isel(y=slice(24, 40))

p = stat.p
rho = stat.RHO[0]
dz = get_dz(rho.z)
data  = data.assign(
    sl=liquid_water_temperature(data.TABS, data.QN, data.QP),
    qt=data.QV + data.QN
)

In [ ]:
X = data[['sl', 'qt', 'SOLIN']]

# mu = X.mean(['x', 'y', 'time'])
# sig2 = ((X - mu)**2).mean()
# sig  = sig2.compute().apply(np.sqrt)

# X = (X-mu)/sig
X = X.stack(batch=['x' ,'y' , 'time'])
X = np.concatenate((X['qt'].T, X['sl'].T, X['SOLIN'].values[:,None]), axis=-1)
y = data.QRAD.stack(batch=['x', 'y','time']).transpose('batch', 'z').values

In [ ]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import Adam


inds = np.random.choice(X.shape[0], 100000)

model = Sequential()
model.add(BatchNormalization(input_shape=(69,)))
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(Dense(34,activation='linear'))

model.compile(loss='mse',
              optimizer=Adam(),
              metrics=['accuracy'])

model.fit(X[inds], y[inds], epochs=5)

In [ ]:
loc = data.isel(x=0, y=8)

Xl = loc[['qt', 'sl', 'SOLIN']]
# X = (X-mu)/sig
Xl = Xl.stack(batch=['time'])

# 
mu = Xl['SOLIN'].mean('batch').values


Xl, mu = xr.broadcast(Xl, Xl.mean('batch'))

# too see which variables are important you can swap the `Xl` values with `mu` below
# when you do this you will see that the 
# 1. SOLIN allows the scheme to get the diurnal cycle
# 2. QT is needed for accurate vertical profile
# 3. SL doesn't matter

Xl_np = np.concatenate((Xl['qt'], Xl['sl'], Xl['SOLIN'].values[:, 0:1]), axis=-1)
yl = loc.QRAD.transpose('time', 'z')

qrad_pred = model.predict(Xl_np)

In [ ]:
plt.pcolormesh(qrad_pred.T)
plt.colorbar()

In [ ]:
plt.scatter(qrad_pred, yl, alpha=.05)
plt.plot((0,7), (0,7), 'k')

In [ ]:
plt.pcolormesh(yl.T)
plt.colorbar()

Here it is with SOLIN fixed at it's mean value

In [ ]:
Xl_np = np.concatenate((Xl['qt'], Xl['sl'], mu['SOLIN'].values[:, 0:1]), axis=-1)
yl = loc.QRAD.transpose('time', 'z')

qrad_pred = model.predict(Xl_np)
plt.pcolormesh(qrad_pred.T)
plt.colorbar()

You can see that the diurnal cycle is gone.

And qt fixed at the mean vlaue

In [ ]:
Xl_np = np.concatenate((mu['qt'], Xl['sl'], Xl['SOLIN'].values[:, 0:1]), axis=-1)
yl = loc.QRAD.transpose('time', 'z')

qrad_pred = model.predict(Xl_np)
plt.pcolormesh(qrad_pred.T)
plt.colorbar()

Ther vertical structure is very unrealistic.

As you can see, we obtain excellent performance. This indicates that we can train a neural network separately for radiation and the convection. Overall, this suggests we should use an objective function like
$$ J(f_s, f_q, r) = \frac{1}{T} \sum_{m=1}^T||F^{m}(\phi^n)  - \phi^{n+m}||^2 + \alpha||r(x^n) -Q_{rad}||^2  
                      + \beta|| c_p \int( f_s(x^n) - r)\frac{dp}{g} - SHF -  LP  ||^2
                      + \gamma||L\int f_q(x^n) \frac{dp}{g} + LHF -  LP  ||^2
$$